In [ ]:

#  Here we demonstrate the following Dask DataFrame techniques;
#
#     .  How to create a Dask DataFrame from an array.
#
#     .  Given a source Dask DataFrame, create a second new DataFrame with just
#        unique values, and the ability to link back to the source.
#
#        We might use this technique when given a flat file of data, that we derive
#        nodes and edges from.


In [1]:

NUM_PARTITIONS  = 3

print("--")


--


In [2]:

import numpy as np
import pandas as pd
   #
import dask.array as da
import dask.dataframe as dd
   #
from dask.dataframe import from_pandas

from tabulate import tabulate

print("--")


--


In [3]:

#  Create our source Dask DataFrame from an array

np_airports = np.array([
       #
   ["MKE", "Milwaukee"     , "WI", "Airport"],
   ["ORD", "Chicago O-Hare", "IL", "Airport"],
   ["SJC", "San Jose"      , "CA", "Airport"],
   ["LAX", "Los Angeles"   , "CA", "Airport"],
   ["DEN", "Denver"        , "CO", "Airport"],
       #
   ], dtype="str")

pd_airports = pd.DataFrame(np_airports, columns = ["airport_code", "airport_name", "state_code", "LABEL"])
   #
dd_airports = from_pandas(pd_airports, npartitions = NUM_PARTITIONS)


for l_each in dd_airports.iterrows():
      #
   l_airport_code   = l_each[1][0]
   l_airport_name   = l_each[1][1]
   l_state_code     = l_each[1][2]
   l_LABEL          = l_each[1][3]
      #
   print("Airport:  %3s   %-18s   %-2s   %-10s" % (l_airport_code, l_airport_name, l_state_code, l_LABEL))

print("--")

#  Sample output,
#
#  Airport:  MKE   Milwaukee            WI   Airport   
#  Airport:  ORD   Chicago O-Hare       IL   Airport   
#  Airport:  SJC   San Jose             CA   Airport   
#  Airport:  LAX   Los Angeles          CA   Airport   
#  Airport:  DEN   Denver               CO   Airport  


Airport:  MKE   Milwaukee            WI   Airport   
Airport:  ORD   Chicago O-Hare       IL   Airport   
Airport:  SJC   San Jose             CA   Airport   
Airport:  LAX   Los Angeles          CA   Airport   
Airport:  DEN   Denver               CO   Airport   
--


#  Write DataFrame to local TXT file  (local to Jupyter Docker container)

In [8]:

#  Write DataFrame to an ASCII file that is local to Jupyter Docker container

dd_airports2 = dd_airports.repartition(npartitions=1) 

dd_airports2.to_csv("42_parent_folder", index=None, sep="|")



['/home/jovyan/work/My_KG_NoteBooks/N4_GNN_MovieDB/06_Compulsaries/42_parent_folder/0.part']

#  Write DataFrame to GS  (easier download later)

In [9]:

#  Settings:
#     Need a public or private accessible storage bucket on GCP or similar ..
#
#  From,
#     https://stackoverflow.com/questions/36314797/write-a-pandas-dataframe-to-google-cloud-storage-or-bigquery
#     https://stackoverflow.com/questions/29325458/dictionary-column-in-pandas-dataframe/29325954#29325954

import os
from google.cloud import storage


#  Setup for all work below ..
#
#  Url for viewing,  https://console.cloud.google.com/storage/browser/farrell-bucket

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/jovyan/work/My_KG_NoteBooks/03_MyKeys.json"

l_bucket = "farrell-bucket"


g_client = storage.Client()
   #
g_bucket = g_client.get_bucket(l_bucket)

print("--")


DefaultCredentialsError: File /home/jovyan/work/03_MyKeys.json was not found.

In [ ]:

dd_airports2 = dd_airports.repartition(npartitions=1) 


l_file = "load_test/node.parquet"
   #
g_bucket.blob(l_file).upload_from_string(dd_airports2.to_parquet(engine="pyarrow", version="2.6"), "application/octet-stream")

l_file = "load_test/node.txt"
   #
g_bucket.blob(l_file).upload_from_string(dd_airports2.to_csv(header=None, index=None, sep="|"), "text/plain")

